<a href="https://colab.research.google.com/github/mmaguero/diploma_fpuna_nlp_ia/blob/master/2025/rag_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Simple Chatbot with LangGraph and Chainlit:
## A Step-by-Step Tutorial

## Prerequisites

In [1]:
# RAG Chatbot
!pip install langgraph langchain-openai chainlit python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━

In [2]:
!mkdir /content/rag_chatbot
%cd /content/rag_chatbot

/content/rag_chatbot


In [9]:
%%writefile .env
OPENROUTER_API_KEY=
OPENROUTER_BASE_URL=https://openrouter.ai/api/v1

Overwriting .env


## Breaking Down the Code

In [10]:
%%writefile chatbot.py
###############
# Initial setup
###############
from langchain_core.messages import HumanMessage, AIMessageChunk
from langchain_core.runnables.config import RunnableConfig
from langchain_openai import ChatOpenAI

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
import chainlit as cl
import os
from dotenv import load_dotenv

import logging
logging.basicConfig(
    filename='app.log',
    level=logging.INFO,  # Log messages of INFO severity and above
    format='%(asctime)s - %(levelname)s - %(message)s' # Optional: format the log messages
)
logging.warning("Saving!")

load_dotenv()  # Load environment variables from a .env file

openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

####################
# Defining the Graph
####################
logging.warning("Defining the Graph!")
workflow = StateGraph(state_schema=MessagesState)

model = ChatOpenAI(
    #model='google/gemini-2.5-flash-lite',
    model='google/gemini-2.0-flash-exp:free',
    #model='google/gemma-3n-e4b-it:free',
    base_url=os.getenv('OPENROUTER_BASE_URL', 'https://openrouter.ai/api/v1'),
    api_key=openrouter_api_key,
    temperature=0,  # Set for deterministic, less creative responses
)

def call_model(state: MessagesState):
    """Invokes the model with the current state and returns the new message."""
    response = model.invoke(state['messages'])
    return {'messages': response}  # Update the state with the model's response

workflow.add_node('model', call_model)  # Add the model-calling function as a node
workflow.add_edge(START, 'model')  # Set the 'model' node as the entry point

######################
# Compliling the Graph
######################
logging.warning("Compliling the Graph!")
memory = MemorySaver()  # Initialize in-memory storage for conversation history

# Compile the graph into a runnable, adding the memory checkpointer
app = workflow.compile(checkpointer=memory)

##############################
# Integrating with Chainlit UI
##############################
logging.warning("Integrating with Chainlit UI!")
@cl.on_message
async def main(message: cl.Message):
    """Process incoming user messages and stream back the AI's response."""
    answer = cl.Message(content='...')  # Create an empty message to stream the response into
    await answer.send()

    # Configure the runnable to associate the conversation with the user's session
    config: RunnableConfig = {'configurable': {'thread_id': cl.context.session.thread_id}}

    # Stream the graph's output
    for msg, _ in app.stream(
        {'messages': [HumanMessage(content=message.content)]},  # Pass the user's message
        config,
        stream_mode='messages',  # Stream individual message chunks
    ):
        # Check if the current streamed item is an AI message chunk
        if isinstance(msg, AIMessageChunk):
            answer.content += msg.content  # type: ignore # Append the content chunk
            await answer.update()  # Update the UI with the appended content

Overwriting chatbot.py


## Running and Testing Your Chatbot

In [11]:
#!chainlit run chatbot.py
!while true; do nohup chainlit run chatbot.py -w >/dev/null 2>&1; sleep 5; done >/dev/null 2>&1 &

### Colab

In [17]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://8000-m-s-3tmib24wemb5m-c.us-west1-1.prod.colab.dev


### localtunnel

In [12]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
changed 22 packages in 3s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [15]:
# password: https://loca.lt/mytunnelpassword
!lt --port 8000 & echo "password " & curl https://loca.lt/mytunnelpassword

password 
104.196.233.114your url is: https://hot-parrots-kiss.loca.lt
